### Melon100 Chart
* 100곡의 노래의 제목과 SongID 추출해서 list에 저장하기
* 100곡 노래의 상세정보를 추출해서 list와 dict에 저장해서 json 파일로 저장하기
* json 파일을 load하여 Pandas의 DataFrame에 저장하기
* DataFrame 객체를 DB의 Table에 저장하기

In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [25]:
url = 'https://www.melon.com/chart/index.htm'
req_header_dict = {
    # 요청헤더 : 브라우저정보
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}
res = requests.get(url, headers=req_header_dict)
print(res.status_code)

if res.ok:
    # 소스보기의 텍스트
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    #<div id="tb_list"><tr><a>
    print(len(soup.select("div#tb_list tr a[href*='playSong']")))
    a_tags = soup.select("div#tb_list tr a[href*='playSong']")

    #노래 100곡의 정보를 저장할 list 선언
    song_list = []
    for idx,a_tag in enumerate(a_tags,1):
        #노래 1곡의 정보를 저장할 dict 선언
        song_dict = {}
        #<a href="">노래제목</a>
        song_title = a_tag.text
        song_dict['song_title'] = song_title

        #a태그의 href 속성의 값을 추출하기 javascript:melon.play.playSong('1000002721',34535898);
        href_value = a_tag['href']
        #Song ID를 찾기 위한 정규표현식
        matched = re.search(r'(\d+)\);', href_value)
        if matched:
            song_id = matched.group(1) # group(0) : 34535898);  group(1) : 34535898
            song_dict['song_id'] = song_id
            song_detail_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
            song_dict['song_detail_url'] = song_detail_url
            #print(song_dict)
            song_list.append(song_dict)

print(len(song_list))
print(song_list[0])

200
100
100


In [38]:
import requests
from bs4 import BeautifulSoup
import re

req_header_dict = {
    # 요청헤더 : 브라우저정보
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}

song_detail_list = []
for idx,song in enumerate(song_list,1):
    #노래1곡의 상세정보를 저장할 dict
    song_detail_dict = {}
    song_detail_url = song['song_detail_url']

    res = requests.get(song_detail_url, headers=req_header_dict)
    print(res.status_code)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        print(idx, song['song_title'])
        song_detail_dict['곡명'] = song['song_title']
        # [<span>임영웅</span>, <span></span>]
        singer_span = soup.select("a[href*='goArtistDetail'] span")
        if singer_span:
            song_detail_dict['가수'] = singer_span[0].text

        #<div class="meta"><dd></dd>
        song_dd = soup.select("div.meta dd")
        if song_dd:
            #print(song_dd[0].text)
            song_detail_dict['앨범'] = song_dd[0].text
            song_detail_dict['발매일'] = song_dd[1].text
            song_detail_dict['장르'] = song_dd[2].text

        song_id = song['song_id']
        like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}'
        like_res = requests.get(like_url, headers=req_header_dict)
        if like_res.ok:
            #print(like_res.status_code)
            #{"contsLike":[{"CONTSID":34061322,"LIKEYN":"N","SUMMCNT":123931}],"httpDomain":"http://www.melon.com","httpsDomain":"https://www.melon.com","staticDomain":"https://static.melon.co.kr"}
            #print(like_res.json()['contsLike'][0]['SUMMCNT'])
            song_detail_dict['좋아요'] = like_res.json()['contsLike'][0]['SUMMCNT']

        #<div class="lyric" id="d_video_summary">
        lyric_div = soup.select("div#d_video_summary")
        if lyric_div:
            lyric_temp = lyric_div[0].text
            # \r\n\t 특수문자를 찾아주는 Pattern 객체생성
            pattern = re.compile(r'[\r\n\t]')
            # \r\n\t 특수문자를 ''(empty string)으로 대체(substitute)해라
            lyric = pattern.sub('', lyric_temp.strip())
        else:
            lyric = ''
        song_detail_dict['가사'] = lyric

        #print(song_detail_dict)
        song_detail_list.append(song_detail_dict)

print(len(song_detail_list))
song_detail_list[:3]

200
1 사랑은 늘 도망가
[<dd><a href="javascript:melon.link.goAlbumDetail('10735654');">신사와 아가씨 OST Part.2</a></dd>, <dd>2021.10.11</dd>, <dd>발라드, 국내드라마</dd>, <dd>Flac 16/24bit</dd>]
123990
200
2 취중고백
[<dd><a href="javascript:melon.link.goAlbumDetail('10816959');">취중고백</a></dd>, <dd>2021.12.19</dd>, <dd>발라드</dd>, <dd>Flac 16bit</dd>]
68684
200
3 호랑수월가
[<dd><a href="javascript:melon.link.goAlbumDetail('10842005');">호랑수월가</a></dd>, <dd>2022.01.15</dd>, <dd>발라드</dd>, <dd>Flac 16bit</dd>]
35285
200
4 Step Back
[<dd><a href="javascript:melon.link.goAlbumDetail('10830791');">Step Back</a></dd>, <dd>2022.01.03</dd>, <dd>댄스</dd>, <dd>Flac 16bit</dd>]
65658
200
5 회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)
[<dd><a href="javascript:melon.link.goAlbumDetail('10775252');">쇼미더머니 10 Episode 2</a></dd>, <dd>2021.11.13</dd>, <dd>랩/힙합</dd>, <dd>Flac 16/24bit</dd>]
186557
200
6 ELEVEN
[<dd><a href="javascript:melon.link.goAlbumDetail('10798794');">ELEVEN</a></dd>, <dd>2021.12.01</dd>, <dd>댄스</dd>, <dd>Flac 16bit</dd

[{'곡명': '사랑은 늘 도망가',
  '가수': '임영웅',
  '앨범': '신사와 아가씨 OST Part.2',
  '발매일': '2021.10.11',
  '장르': '발라드, 국내드라마',
  '좋아요': 123990,
  '가사': '눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨었지내 아름답던 사람아사랑이란 게 참 쓰린 거더라잡으려 할수록 더 멀어지더라이별이란 게 참 쉬운 거더라내 잊지 못할 사람아사랑아 왜 도망가수줍은 아이처럼행여 놓아버릴까 봐꼭 움켜쥐지만그리움이 쫓아 사랑은 늘 도망가잠시 쉬어가면 좋을 텐데바람이 분다 옷깃을 세워도차가운 이별의 눈물이 차올라잊지 못해서 가슴에 사무친내 소중했던 사람아사랑아 왜 도망가수줍은 아이처럼행여 놓아버릴까 봐꼭 움켜쥐지만그리움이 쫓아 사랑은 늘 도망가잠시 쉬어가면 좋을 텐데기다림도 애태움도 다 버려야 하는데무얼 찾아 이 길을 서성일까무얼 찾아 여기 있나사랑아 왜 도망가수줍은 아이처럼행여 놓아버릴까 봐꼭 움켜쥐지만그리움이 쫓아 사랑은 늘 도망가잠시 쉬어가면 좋을 텐데잠시 쉬어가면 좋을 텐데'},
 {'곡명': '취중고백',
  '가수': '김민석 (멜로망스)',
  '앨범': '취중고백',
  '발매일': '2021.12.19',
  '장르': '발라드',
  '좋아요': 68684,
  '가사': '뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서 오랜만에술을 좀 했는데자꾸만 니 얼굴 떠올라무작정 달려왔어이 맘 모르겠니요즘 난 미친 사람처럼너만 생각해대책없이 네가 점점 좋아져아냐 안 취했어 진짜야널 정말 사랑해눈물이 날만큼 원하고 있어정말로 몰랐니가끔 전화해 장난치듯주말엔 뭐할거냐며너의 관심 끌던 나를그리고 한번씩 누나 주려 샀는데너 그냥 준다고생색 낸 선물도 너 때문에 산거야이 맘 모르겠니요즘 난 미친 사람처럼너만 생각해대책없이 네가 점점 좋아져아냐 안 취했어 진짜야널 정말 사랑해진심이야 믿어줘갑자기 이런 말 놀랐다면 미안해부담이 되는

In [44]:
import json

with open('data/songs.json','w', encoding='utf-8') as file:
    json.dump(song_detail_list, file)

In [45]:
with open('data/songs.json',encoding='utf-8') as file:
    song_json= json.loads(file.read())

#print(song_json)

### Pandas 의 DataFrame 객체 사용하기

In [53]:
#song_detail_list을 읽어서 DataFrame 객체를 생성하는 방법
#DataFrame 객체를 생성
import pandas as pd

song_df2 = pd.DataFrame(columns=['곡명','가수','앨범','발매일','장르','좋아요','가사'])

#1개의 row = Series 객체, 1개의 column = Series 객체
for song_detail in song_detail_list:
    # dict <=> Series
    series_obj = pd.Series(song_detail)
    # list <=> DataFrame
    song_df2 = song_df2.append(series_obj, ignore_index=True)

song_df2.head()

<class 'pandas.core.frame.DataFrame'>


,곡명,가수,앨범,발매일,장르,좋아요,가사
0,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",123990,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...
1,취중고백,김민석 (멜로망스),취중고백,2021.12.19,발라드,68684,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...
2,호랑수월가,탑현,호랑수월가,2022.01.15,발라드,35285,흐르는 저 하늘을 물어 채는 범처럼태산에 날아들어 숨어드는 새처럼동산을 뛰고 뛰어가...
3,Step Back,GOT the beat,Step Back,2022.01.03,댄스,65658,You must step back어델 어델 봐너 감히 누구라고 날 제껴이쯤에서 물러...
4,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo,쇼미더머니 10 Episode 2,2021.11.13,랩/힙합,186557,내가 슬플 때마다이 노래가 찾아와세상이 둥근 것처럼 우린 동글동글인생은 회전목마우린...


In [55]:

import pandas as pd

#json file을 읽어서 DataFrame 객체를 생성하는 방법
song_df = pd.read_json('data/songs.json')
print(type(song_df))
song_df.head()

<class 'pandas.core.frame.DataFrame'>


,곡명,가수,앨범,발매일,장르,좋아요,가사
0,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",123990,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...
1,취중고백,김민석 (멜로망스),취중고백,2021.12.19,발라드,68684,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...
2,호랑수월가,탑현,호랑수월가,2022.01.15,발라드,35285,흐르는 저 하늘을 물어 채는 범처럼태산에 날아들어 숨어드는 새처럼동산을 뛰고 뛰어가...
3,Step Back,GOT the beat,Step Back,2022.01.03,댄스,65658,You must step back어델 어델 봐너 감히 누구라고 날 제껴이쯤에서 물러...
4,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo,쇼미더머니 10 Episode 2,2021.11.13,랩/힙합,186557,내가 슬플 때마다이 노래가 찾아와세상이 둥근 것처럼 우린 동글동글인생은 회전목마우린...


In [64]:
song_df.tail()

,곡명,가수,앨범,발매일,장르,좋아요,가사
95,좋아좋아,조정석,슬기로운 의사생활 시즌2 OST Part 5,2021.07.16,"발라드, 국내드라마",83250,처음 널 만나는 날노란 세 송이 장미를 들고룰루랄라 신촌을 향하는내 가슴은 마냥 두...
96,나 그댈위해 시 한편을 쓰겠어,케이시 (Kassy),옛이야기,2021.10.28,발라드,31440,나 그댈 위해 시 한 편을 쓰겠어나 그대에게 못다 한 얘기 많아차마 그때 하지 못 ...
97,깨우지 마,MC몽,X by X [ 꿈 ],2022.01.21,랩/힙합,7704,깨우지 마 꿈속으로도망치듯 잠에 든 날하루 중 내가 유일하게무책임해도 되는 시간날 ...
98,Still I Love You,토요태,Still I Love You,2021.12.18,R&B/Soul,31471,여보세요어 잘 지내지오빠 전화하지 말랬잖아아 그치 어우 야 밖에 많이 춥다따뜻하게 ...
99,MBTI (Feat. 쿠기 & 로꼬) (Prod. GRAY),BE'O (비오),쇼미더머니 10 Semi Final,2021.11.27,랩/힙합,45093,Million 내 몸값을 더 올려Billion 내 몸값을 더 올려Trillion 내...


In [62]:
print('shape', song_df.shape)
print('columns', song_df.columns)
print('index', song_df.index)
print('values', type(song_df.values)) #2차원배열

In [63]:
song_df.info()

,곡명,가수,앨범,발매일,장르,좋아요
0,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",123990
10,이제 나만 믿어요,임영웅,내일은 미스터트롯 우승자 특전곡,2020.04.03,성인가요,146420
17,다시 사랑한다면 (김필 Ver.),임영웅,사랑의 콜센타 PART56,2021.05.25,성인가요,40774
18,그대라는 사치,임영웅,사랑의 콜센타 PART62,2021.07.06,발라드,36831
23,별빛 같은 나의 사랑아,임영웅,별빛 같은 나의 사랑아,2021.03.09,성인가요,69703
25,잊었니,임영웅,사랑의 콜센타 PART60,2021.06.22,성인가요,36020
28,HERO,임영웅,HERO,2020.11.04,성인가요,69936
29,끝사랑,임영웅,사랑의 콜센타 PART61,2021.06.29,성인가요,33317
32,Bk Love,임영웅,뽕숭아학당 PART23,2021.06.21,랩/힙합,33012
42,계단말고 엘리베이터,임영웅,계단말고 엘리베이터,2018.08.20,성인가요,56703


In [65]:
# 가수 컬럼의 값을 선택하기
print(type(song_df['가수']))
song_df['가수'].head()

# 가수 컬럼의 값별로 Row Counting
song_df['가수'].value_counts()

# unique한(중복제거) 가수 컬럼의 값을 가져오기
song_df['가수'].unique()

song_df['장르'].unique()

# 가사 empty string check
song_df.loc[song_df['가사'] == '']

# 방탄소년단 노래 선택하기
song_df['가수'] == '방탄소년단'

song_df.loc[song_df['가수'] == '방탄소년단']

<class 'pandas.core.series.Series'>


,곡명,가수,앨범,발매일,장르,좋아요,가사
35,Butter,방탄소년단,Butter,2021.05.21,댄스,263255,Smooth like butterLike a criminal undercoverGo...
41,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스,428692,Cos ah ahI’m in the stars tonightSo watch me b...
45,Permission to Dance,방탄소년단,Butter / Permission to Dance,2021.07.09,댄스,195014,It’s the thought of being youngWhen your heart...
77,봄날,방탄소년단,YOU NEVER WALK ALONE,2017.02.13,랩/힙합,555843,보고 싶다이렇게 말하니까 더 보고 싶다너희 사진을 보고 있어도보고 싶다너무 야속한 ...
82,작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey),방탄소년단,MAP OF THE SOUL : PERSONA,2019.04.12,댄스,431668,모든 게 궁금해How’s your dayOh tell me뭐가 널 행복하게 하는지O...
94,Life Goes On,방탄소년단,BE,2020.11.20,랩/힙합,233287,어느 날 세상이 멈췄어아무런 예고도 하나 없이봄은 기다림을 몰라서눈치 없이 와버렸어...


In [64]:
def get_song(singer):
    return song_df.loc[song_df['가수'] == singer, '곡명':'좋아요']

In [ ]:
get_song('임영웅')

#### 특정 행과 열을 선택하기
* loc[], iloc[] 사용한다.
1. Slicing 을 사용함으로 구간을 주어서 행과 열을 선택
2. <pre>[ ](list) 를 사용해서 특정행과 열을 선택</pre>
3. 조건식을 만족하는 행과 열을 선택

In [67]:
# Slicing : 인덱스가 0부터 5까지의 행과 모든 열을 선택하기
song_df.loc[0:5]      # song_df.loc[0:5,:]

,곡명,가수,앨범,발매일,장르,좋아요,가사
0,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",123990,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...
1,취중고백,김민석 (멜로망스),취중고백,2021.12.19,발라드,68684,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...
2,호랑수월가,탑현,호랑수월가,2022.01.15,발라드,35285,흐르는 저 하늘을 물어 채는 범처럼태산에 날아들어 숨어드는 새처럼동산을 뛰고 뛰어가...
3,Step Back,GOT the beat,Step Back,2022.01.03,댄스,65658,You must step back어델 어델 봐너 감히 누구라고 날 제껴이쯤에서 물러...
4,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo,쇼미더머니 10 Episode 2,2021.11.13,랩/힙합,186557,내가 슬플 때마다이 노래가 찾아와세상이 둥근 것처럼 우린 동글동글인생은 회전목마우린...
5,ELEVEN,IVE (아이브),ELEVEN,2021.12.01,댄스,103233,따분한 나의 눈빛이무표정했던 얼굴이널 보며 빛나고 있어널 담은 눈동자는 odd내 안...


In [69]:
# Slicing : 인덱스가 0부터 5까지의 행과 모든 열을 선택하기, 1개의 행르 skip
song_df.loc[0:10:2]      # song_df.loc[0:10:2,:]

,곡명,가수,앨범,발매일,장르,좋아요,가사
0,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",123990,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...
2,호랑수월가,탑현,호랑수월가,2022.01.15,발라드,35285,흐르는 저 하늘을 물어 채는 범처럼태산에 날아들어 숨어드는 새처럼동산을 뛰고 뛰어가...
4,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo,쇼미더머니 10 Episode 2,2021.11.13,랩/힙합,186557,내가 슬플 때마다이 노래가 찾아와세상이 둥근 것처럼 우린 동글동글인생은 회전목마우린...
6,리무진 (Feat. MINO) (Prod. GRAY),BE'O (비오),쇼미더머니 10 Episode 3,2021.11.20,랩/힙합,162366,까만 리무진 보며꿈을 키웠지언젠가는 나도 저걸 갖게 될 거야커다란 리무진에서 내가 ...
8,Dreams Come True,aespa,Dreams Come True - SM STATION,2021.12.20,댄스,74889,"Uh, You wanna feel the vibeThat I’m feeling an..."
10,이제 나만 믿어요,임영웅,내일은 미스터트롯 우승자 특전곡,2020.04.03,성인가요,146420,무얼 믿은 걸까 부족했던 내게서나조차 못 믿던 내게 여태 머문 사람무얼 봤던 걸까 ...


In [101]:
# Slicing : 인덱스가 0부터 5까지의 행과 곡명부터 장르까지의 열을 선택하기
song_df.loc[0:5,'곡명':'장르']

,곡명,가수,앨범,발매일,장르
0,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마"
1,취중고백,김민석 (멜로망스),취중고백,2021.12.19,발라드
2,호랑수월가,탑현,호랑수월가,2022.01.15,발라드
3,Step Back,GOT the beat,Step Back,2022.01.03,댄스
4,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo,쇼미더머니 10 Episode 2,2021.11.13,랩/힙합
5,ELEVEN,IVE (아이브),ELEVEN,2021.12.01,댄스


In [73]:
# 여러개의 0,4,7,10 행과 열을 선택
song_df.loc[[0,4,7,10]]

,곡명,가수,앨범,발매일,장르,좋아요,가사
0,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",123990,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...
4,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo,쇼미더머니 10 Episode 2,2021.11.13,랩/힙합,186557,내가 슬플 때마다이 노래가 찾아와세상이 둥근 것처럼 우린 동글동글인생은 회전목마우린...
7,Counting Stars (Feat. Beenzino),BE'O (비오),Counting Stars,2021.12.12,랩/힙합,147753,Counting stars밤하늘에 펄Better than your LVYour LV...
10,이제 나만 믿어요,임영웅,내일은 미스터트롯 우승자 특전곡,2020.04.03,성인가요,146420,무얼 믿은 걸까 부족했던 내게서나조차 못 믿던 내게 여태 머문 사람무얼 봤던 걸까 ...


In [76]:
# 여러개의 0,4,7,10 행과 곡명, 가수, 장르 열을 선택
song_df.loc[[0,4,7,10],['곡명','가수','장르']]

,곡명,가수,장르
0,사랑은 늘 도망가,임영웅,"발라드, 국내드라마"
4,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo,랩/힙합
7,Counting Stars (Feat. Beenzino),BE'O (비오),랩/힙합
10,이제 나만 믿어요,임영웅,성인가요


In [78]:
# 여러개의 0,4,7,10 행과 좋아요 열만 선택
song_df.loc[[0,4,7,10],['좋아요']]

,좋아요
0,123990
4,186557
7,147753
10,146420


In [82]:
# 10부터 20까지의 행과 곡명, 가수, 좋아요 열을 선택

song_df.loc[10:20,['곡명','가수','좋아요']]

,곡명,가수,좋아요
10,이제 나만 믿어요,임영웅,146420
11,Can't Control Myself,태연 (TAEYEON),47543
12,겨울잠,아이유,89258
13,다정히 내 이름을 부르면,경서예지,109305
14,신호등,이무진,256451
15,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. TOIL),베이식 (Basick),107153
16,STAY,The Kid LAROI,202790
17,다시 사랑한다면 (김필 Ver.),임영웅,40774
18,그대라는 사치,임영웅,36831
19,Next Level,aespa,235360


In [85]:
# 장르가 댄스인 행과 모든열을 선택

song_df.loc[song_df['장르'] == '댄스']

,곡명,가수,앨범,발매일,장르,좋아요,가사
3,Step Back,GOT the beat,Step Back,2022.01.03,댄스,65658,You must step back어델 어델 봐너 감히 누구라고 날 제껴이쯤에서 물러...
5,ELEVEN,IVE (아이브),ELEVEN,2021.12.01,댄스,103233,따분한 나의 눈빛이무표정했던 얼굴이널 보며 빛나고 있어널 담은 눈동자는 odd내 안...
8,Dreams Come True,aespa,Dreams Come True - SM STATION,2021.12.20,댄스,74889,"Uh, You wanna feel the vibeThat I’m feeling an..."
19,Next Level,aespa,Next Level,2021.05.17,댄스,235360,I’m on the Next Level Yeah절대적 룰을 지켜내 손을 놓지 말아결...
21,SMILEY (Feat. BIBI),YENA (최예나),ˣ‿ˣ (SMiLEY),2022.01.17,댄스,44487,울지 마 울지 마어린아이같이웃는 게 웃는 게이기는 거라구Youre so cuteYo...
26,Savage,aespa,Savage - The 1st Mini Album,2021.10.05,댄스,139539,Oh my gosh!Don't you know I’m a Savage?I’m a K...
35,Butter,방탄소년단,Butter,2021.05.21,댄스,263255,Smooth like butterLike a criminal undercoverGo...
41,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스,428692,Cos ah ahI’m in the stars tonightSo watch me b...
45,Permission to Dance,방탄소년단,Butter / Permission to Dance,2021.07.09,댄스,195014,It’s the thought of being youngWhen your heart...
50,Weekend,태연 (TAEYEON),Weekend,2021.07.06,댄스,159195,가장 가까운 바다혼자만의 영화관그냥 이끌리는 대로 해도 괜찮으니까Every morn...


In [88]:
# 장르가 댄스인 행과 곡명 부터 장르까지의 열을 선택
song_df.loc[song_df['장르'] == '댄스', '곡명':'장르'].reset_index(drop=True)

,곡명,가수,앨범,발매일,장르
0,Step Back,GOT the beat,Step Back,2022.01.03,댄스
1,ELEVEN,IVE (아이브),ELEVEN,2021.12.01,댄스
2,Dreams Come True,aespa,Dreams Come True - SM STATION,2021.12.20,댄스
3,Next Level,aespa,Next Level,2021.05.17,댄스
4,SMILEY (Feat. BIBI),YENA (최예나),ˣ‿ˣ (SMiLEY),2022.01.17,댄스
5,Savage,aespa,Savage - The 1st Mini Album,2021.10.05,댄스
6,Butter,방탄소년단,Butter,2021.05.21,댄스
7,Dynamite,방탄소년단,Dynamite (DayTime Version),2020.08.24,댄스
8,Permission to Dance,방탄소년단,Butter / Permission to Dance,2021.07.09,댄스
9,Weekend,태연 (TAEYEON),Weekend,2021.07.06,댄스


In [92]:
# '좋아요' 건수가 평균보다 높은 행을 선택

like_mean = song_df['좋아요'].mean()
print(like_mean)

print('like Max',song_df['좋아요'].max())
print('like Min',song_df['좋아요'].min())
print('like Median',song_df['좋아요'].median())

117363.55
like Max 555843
like Min 3130
like Median 79254.0


In [94]:
like_max = song_df['좋아요'].max()
song_df.loc[song_df['좋아요'] == like_max]

,곡명,가수,앨범,발매일,장르,좋아요,가사
77,봄날,방탄소년단,YOU NEVER WALK ALONE,2017.02.13,랩/힙합,555843,보고 싶다이렇게 말하니까 더 보고 싶다너희 사진을 보고 있어도보고 싶다너무 야속한 ...


In [97]:
#좋아요 건수가 평균보다 높거나 같은 행과 곡명,가수,좋아요 열을 선택하고 index를 reset 하기

song_df.loc[song_df['좋아요'] >= like_mean, ['곡명','가수','좋아요']].reset_index(drop=True)

,곡명,가수,좋아요
0,사랑은 늘 도망가,임영웅,123990
1,"회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)",sokodomo,186557
2,리무진 (Feat. MINO) (Prod. GRAY),BE'O (비오),162366
3,Counting Stars (Feat. Beenzino),BE'O (비오),147753
4,이제 나만 믿어요,임영웅,146420
5,신호등,이무진,256451
6,STAY,The Kid LAROI,202790
7,Next Level,aespa,235360
8,strawberry moon,아이유,179840
9,Savage,aespa,139539


In [99]:
#좋아요 건수가 평균보다 높거나 같은 행과 곡명,가수,좋아요 열을 선택하고 좋아요 건수가 높은 순서대로 정렬하고, index를 reset 하기

song_df.loc[song_df['좋아요'] >= like_mean, ['곡명','가수','좋아요']].sort_values(by='좋아요')

,곡명,가수,좋아요
0,사랑은 늘 도망가,임영웅,123990
92,멜로디,ASH ISLAND,125376
62,"사이렌 Remix (Feat. UNEDUCATED KID, Paul Blanco)",호미들,130256
26,Savage,aespa,139539
10,이제 나만 믿어요,임영웅,146420
7,Counting Stars (Feat. Beenzino),BE'O (비오),147753
49,My Universe,Coldplay,147798
78,Dun Dun Dance,오마이걸 (OH MY GIRL),151767
50,Weekend,태연 (TAEYEON),159195
6,리무진 (Feat. MINO) (Prod. GRAY),BE'O (비오),162366


In [100]:
#좋아요 건수가 평균보다 높은 행과 곡명,가수,좋아요 열을 선택하고 좋아요 건수가 높은 순서대로 정렬하고, index를 reset 하기
song_df.loc[song_df['좋아요'] >= like_mean,['곡명','가수','좋아요']].sort_values(by='좋아요',ascending=False).reset_index(drop=True)

,곡명,가수,좋아요
0,봄날,방탄소년단,555843
1,작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey),방탄소년단,431668
2,Dynamite,방탄소년단,428692
3,"모든 날, 모든 순간 (Every day, Every Moment)",폴킴,385891
4,"어떻게 이별까지 사랑하겠어, 널 사랑하는 거지",AKMU (악뮤),373840
5,Blueming,아이유,328419
6,Celebrity,아이유,312340
7,롤린 (Rollin'),브레이브걸스,290787
8,Butter,방탄소년단,263255
9,신호등,이무진,256451
